# 1) Install & Imports

In [1]:

! pip install -q transformers==4.53.3 langchain gspread google-auth pandas fastapi uvicorn pyngrok accelerate

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
from collections import Counter
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import re
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 80.9 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 95.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmanager 0.7.1 requires sqlalchemy==1.2.19, but you have sqlalchemy 2.0.45 which is incompatible.


2026-01-19 19:13:33.201849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768850013.393017      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768850013.448649      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768850013.916626      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768850013.916670      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768850013.916673      55 computation_placer.cc:177] computation placer alr

# 2) Sentiment Analysis Model


In [2]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


# 3) Load LLM (Mistral)

In [3]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
def generate_text(prompt, max_length=15000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

# 4) Custom LangChain LLM

In [5]:
class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop=None) -> str:
        return generate_text(prompt)
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

# 5) Chain → Extract Main Problem


In [6]:
problem_prompt = PromptTemplate(
    input_variables=["review"],
    template="""
        You are an expert customer complaint analyst.

        From the following NEGATIVE review, extract ONLY ONE main problem
        as a very short clear phrase.

        Return ONLY the problem text, nothing else.

        Review:
        "{review}"
"""
)

problem_chain = LLMChain(llm=llm, prompt=problem_prompt)

/tmp/ipykernel_55/3945071181.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  problem_chain = LLMChain(llm=llm, prompt=problem_prompt)


# 6) Output Parser Schema


In [7]:
Problem_schema = ResponseSchema(
    name="Problem",
    description="The main recurring problem."
)

total_occurrences_schema = ResponseSchema(
    name="Number of occurrences",
    description="How many customers reported this problem."
)

severity_level_schema = ResponseSchema(
    name="Severity level",
    description="Severity from 1 (minor) to 5 (critical)."
)

solution_prompt = ResponseSchema(
    name='Suggested Solution',
    description="As an expert customer service manager, suggest the best way to resolve the following problem ,Provide a concise resolution plan."
    )

response_schemas = [
    Problem_schema,
    total_occurrences_schema,
    severity_level_schema,
    solution_prompt
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 7) Chain → Severity + Structured Output


In [8]:
final_prompt_template = """
    You are an expert customer experience analyst.

    Based on the following recurring problem and statistics, generate a structured report.

    Problem: {problem}
    Number of occurrences: {count}

    Decide the severity level from 1 to 5:
    1 = very minor
    3 = moderate
    5 = critical (safety, payment, trust, delivery loss)

    suggest a concise resolution plan to address this problem effectively.

    Return ONLY valid JSON following this schema between '```json and ```'.

    {format_instructions}
"""

final_prompt = PromptTemplate(
    input_variables=["problem", "count", "format_instructions"],
    template=final_prompt_template
)

final_chain = LLMChain(llm=llm, prompt=final_prompt)

# 8) extract the last JSON block from a text

In [9]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

# 9) Google Sheet Connection

In [10]:
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file(
    "/kaggle/input/storied-mantra-457512-f7-5c671f415114/storied-mantra-457512-f7-5c671f415114.json",
    scopes=SCOPES
)

client = gspread.authorize(creds)

sheet = client.open_by_url("https://docs.google.com/spreadsheets/d/19_axzwI81tKb3vZazRHy8KQnTAZeQajal6MKhOl6zr4/edit?gid=0#gid=0").sheet1

# 10) Main System

In [11]:
def main():
        
    data = sheet.get_all_records()
    reviews = [record['Reviews'] for record in data]



    # ---- Step 1: Sentiment filtering
    negative_reviews = []
    for r in reviews:
        result = sentiment_pipeline(r)[0]
        if result["label"] == "NEGATIVE":
            negative_reviews.append(r)

    print("Negative Reviews:", len(negative_reviews))

    # ---- Step 2: Extract problems
    problems = []
    for review in negative_reviews:
        p = problem_chain.run(review=review).strip()
        problems.append(p)

    print("\nExtracted Problems:")
    print(problems)

    # ---- Step 3: Count recurring problems
    counter = Counter(problems)
    most_common_problem, count = counter.most_common(1)[0]

    print("\nMost Common Problem:", most_common_problem)
    print("Occurrences:", count)

    final_results = []

    for problem, count in counter.items():

        final_response = final_chain.run(
            problem=problem,
            count=count,
            format_instructions=format_instructions
        )

        json_text = extract_json_block(final_response)

        output_data = output_parser.parse(json_text)
        final_results.append(output_data)

    print("\n✅ Final Structured Output:\n")
    for result in final_results:
        print(result)
    
    return final_results

In [29]:
main()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Negative Reviews: 4


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Extracted Problems:
['You are an expert customer complaint analyst.\n\n        From the following NEGATIVE review, extract ONLY ONE main problem\n        as a very short clear phrase.\n\n        Return ONLY the problem text, nothing else.\n\n        Review:\n        "The product arrived damaged and customer service ignored me."\n        Problem:', 'You are an expert customer complaint analyst.\n\n        From the following NEGATIVE review, extract ONLY ONE main problem\n        as a very short clear phrase.\n\n        Return ONLY the problem text, nothing else.\n\n        Review:\n        "My order came broken and no one replied to my emails."\n        ----> "Order arrived broken"', 'You are an expert customer complaint analyst.\n\n        From the following NEGATIVE review, extract ONLY ONE main problem\n        as a very short clear phrase.\n\n        Return ONLY the problem text, nothing else.\n\n        Review:\n        "Worst experience ever, the item was damaged."\n        Probl

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



✅ Final Structured Output:

{'Problem': 'Product damage', 'Number of occurrences': '1', 'Severity level': '3', 'Suggested Solution': 'Implement a more robust packaging process and provide clear instructions to customers on how to inspect the product upon delivery. Additionally, train customer service representatives to immediately address and escalate damaged product complaints.'}
{'Problem': 'Order arrived broken', 'Number of occurrences': '1', 'Severity level': '3', 'Suggested Solution': 'Implement a rigorous quality check process before dispatching. Train staff to handle products with care and provide clear instructions on packaging. Offer immediate replacement or refund if the customer reports a broken item.'}
{'Problem': 'Payment gateway failure', 'Number of occurrences': '2', 'Severity level': '5', 'Suggested Solution': 'Test and optimize payment gateway integration, provide alternative payment methods, and offer assistance to customers experiencing payment issues.'}
{'Problem':

[{'Problem': 'Product damage',
  'Number of occurrences': '1',
  'Severity level': '3',
  'Suggested Solution': 'Implement a more robust packaging process and provide clear instructions to customers on how to inspect the product upon delivery. Additionally, train customer service representatives to immediately address and escalate damaged product complaints.'},
 {'Problem': 'Order arrived broken',
  'Number of occurrences': '1',
  'Severity level': '3',
  'Suggested Solution': 'Implement a rigorous quality check process before dispatching. Train staff to handle products with care and provide clear instructions on packaging. Offer immediate replacement or refund if the customer reports a broken item.'},
 {'Problem': 'Payment gateway failure',
  'Number of occurrences': '2',
  'Severity level': '5',
  'Suggested Solution': 'Test and optimize payment gateway integration, provide alternative payment methods, and offer assistance to customers experiencing payment issues.'},
 {'Problem': 'Da

# 11) END_Point_API

In [17]:
# kill the ngrok process if running
ngrok.kill()

In [18]:
NGROK_TOKEN = "33N5iIyI0DDMRt0vMWzjkQ7G61b_YjVQYahGWxy8Xds3ZsVw"
API_KEY = "secret123"

In [19]:

app = FastAPI()

@app.post("/show_reviews")
async def gen(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    return {
        "response": main()
    }

In [20]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://tamiko-unidealized-discernibly.ngrok-free.dev


INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:58245 (Press CTRL+C to quit)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Negative Reviews: 4


In [3]:
import requests
headers = {"Authorization": "Bearer secret123"}
API_URL = "https://tamiko-unidealized-discernibly.ngrok-free.dev/show_reviews"
   
res = requests.post(API_URL, headers=headers, verify=False)
print("STATUS:", res.status_code)
print("RAW RESPONSE:\n", res.text)

data = res.json()

print(data)

c:\Users\abdoa\anaconda3\envs\ml\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tamiko-unidealized-discernibly.ngrok-free.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


STATUS: 200
RAW RESPONSE:
 {"response":[{"Problem":"Late or missing delivery","Number of occurrences":"1000","Severity level":"5","Suggested Solution":"Implement a robust tracking system, offer real-time updates to customers, and provide clear communication on expected delivery times. Additionally, establish a reliable process for resolving missing packages promptly."},{"Problem":"Cold food, slow service","Number of occurrences":"1","Severity level":"3","Suggested Solution":"Implement a new system to track food temperature and service speed in real-time. Train staff to prioritize speed and quality. Offer free drinks or appetizers to customers who experience long wait times."}]}
{'response': [{'Problem': 'Late or missing delivery', 'Number of occurrences': '1000', 'Severity level': '5', 'Suggested Solution': 'Implement a robust tracking system, offer real-time updates to customers, and provide clear communication on expected delivery times. Additionally, establish a reliable process for